In [1]:
!pip3 install -U torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116
!pip install git+https://github.com/brianbt/btorch

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
  Cloning https://github.com/brianbt/btorch to /tmp/pip-req-build-xw2tpa1r
  Running command git clone --filter=blob:none --quiet https://github.com/brianbt/btorch /tmp/pip-req-build-xw2tpa1r
  Resolved https://github.com/brianbt/btorch to commit af13b057b343a724ab9ab6d5baa685c1d62fc54b
  Preparing metadata (setup.py) ... done


In [2]:
# !nvcc --version

In [3]:
import torch
import torchvision
from torchvision import transforms, datasets
from matplotlib import pyplot as plt

import btorch
import btorch.nn as nn
import btorch.nn.functional as F
from btorch.vision import models
from btorch.vision.utils import pplot, img_MinMaxScaler
from btorch.utils.trainer import get_lr

from tqdm import tqdm

# Load Dataset, CIFAR10

In [4]:
# Load CIFAR10 dataset, do augmentation on the trainset
class MultiTransform():
    def __init__(self, transforms, n=2):
        self.transforms = transforms
        self.n = n
    def __call__(self, x):
        return [self.transforms(x) for _ in range(self.n)]

transform_train = transforms.Compose([
  transforms.RandomResizedCrop(size=64),
  transforms.RandomHorizontalFlip(),
  transforms.RandomApply([transforms.ColorJitter(0.8, 0.8, 0.8, 0.2)], p=0.8),
  transforms.RandomGrayscale(p=0.2),
  transforms.ToTensor(),
  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
  ])
transform_test = transforms.Compose([
  transforms.Resize(64),
  transforms.ToTensor(),
  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
  ])
trainset = datasets.CIFAR10('./cifar10',train=True, download=True, transform=MultiTransform(transform_train))
testset = datasets.CIFAR10('./cifar10',train=False, download=True, transform=transform_test)

transform_clf_train = transforms.Compose([
  transforms.Resize(64),
  transforms.RandomHorizontalFlip(),
  transforms.ToTensor(),
  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
  ])
trainset_clf = datasets.CIFAR10('./cifar10',train=True, download=True, transform=transform_clf_train)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [5]:
# for i in trainset:
#   pplot(img_MinMaxScaler(torch.stack(i[0])))
#   # pplot(img_MinMaxScaler(i[0][0]))
#   # pplot(img_MinMaxScaler(i[0][1]))
#   break

In [6]:
# for i in train_loader:
#   print(i[0][0].shape)
#   break
# torch.concat(i[0]).shape

# Define InfoNCE Loss

In [7]:
def info_nce_loss(views, temperature=0.07):
    """edit from https://github.com/sthalles/SimCLR/blob/master/simclr.py#L26

    Args:
        views(List[Tensor]): must have at least two elements
        temperature(float): temperature

    Returns:
        Tensor(N*n_views, n_views-1), Tensor(N*n_views,)
      
    Note:
        You should do CrossEntropyLoss() on the output logit and labels

    Examples:
        >>> features = model(x)
        >>> features_aug = model(x_aug)
        >>> logits, labels = info_nce_loss([features, features_aug])
        >>> loss = lossfn(logits, labels)
        >>> loss.backward()
    """
    assert len(views) >= 2
    n_views = len(views)
    batch_size = views[0].shape[0]
    features = torch.cat(views, dim=0)

    labels = torch.cat([torch.arange(batch_size) for i in range(n_views)], dim=0) #(batch_size*n_views)
    labels = (labels.unsqueeze(0) == labels.unsqueeze(1)).float() #(batch_size*n_views)
    labels = labels.to(views[0].device)
    # Below two line is same as cos_similarity 
    features = F.normalize(features, dim=1) #(N*n_views,D)
    similarity_matrix = torch.matmul(features, features.T) #(N*n_views,D)

    # assert similarity_matrix.shape == (
    #     self.args.n_views * self.args.batch_size, self.args.n_views * self.args.batch_size)
    # assert similarity_matrix.shape == labels.shape

    # discard the main diagonal from both: labels and similarities matrix
    mask = torch.eye(labels.shape[0], dtype=torch.bool, device=views[0].device) #(N*n_views*N*n_views), diagonal matrix
    labels = labels[~mask].view(labels.shape[0], -1)#(N*n_views,N*n_views-N), labels without diagonal
    similarity_matrix = similarity_matrix[~mask].view(similarity_matrix.shape[0], -1)#(N*n_views,N*n_views-N), sim_matrix without diagonal
    # Above labels and similarity is the one without diagonal, without self, only the positive-pair remain

    # assert similarity_matrix.shape == labels.shape
    # select and combine multiple positives
    positives = similarity_matrix[labels.bool()].view(labels.shape[0], -1) #(N*n_view,n_view-1)
    # select only the negatives the negatives
    negatives = similarity_matrix[~labels.bool()].view(similarity_matrix.shape[0], -1) #(N*n_view,N*n_view-n_view)

    logits = torch.cat([positives, negatives], dim=1) #(N*n_view,N*n_view-1)
    # Now each data point have (N*n_view-1) values
    # The positive pair's similarity is in the first place
    # Therefore in labels we set all as `0`, refering to the first index.
    labels = torch.zeros(logits.shape[0], dtype=torch.long, device=views[0].device) #(N*n_view)
    
    logits = logits / temperature
    return logits, labels

# Create SimCLR Module

This step is to train a Resnet50 model in constractive learning way.  
So that the Resnet50 backbone(exclude fc layers) will output a emebedding that is similar for each class.

In [8]:
class SimCLR(nn.Module):
    def __init__(self, embedding_dim):
        super(SimCLR, self).__init__()
        self.embedding_dim = embedding_dim
        self.backbone = models.resnet50()
        backbone_out_dim = self.backbone.fc.in_features
        self.backbone.fc = nn.Sequential(nn.Linear(backbone_out_dim, backbone_out_dim),
                                         nn.ReLU(),
                                         nn.Linear(backbone_out_dim, embedding_dim))
    def forward(self, x):
        return self.backbone(x)
    
    @classmethod
    def train_epoch(cls, net, criterion, trainloader, optimizer, epoch_idx, device='cuda', config=None, **kwargs):
        """This is the very basic training function for one epoch. Override this function when necessary

        Returns:
            (float or dict): train_loss
        """
        net.train()
        train_loss = 0
        pbar = tqdm(enumerate(trainloader), total=len(trainloader),
                    disable=(kwargs.get("verbose", 1) == 0))
        batch_idx = 1
        for batch_idx, (inputs, _) in pbar:
            inputs_1 = inputs[0].to(device)
            inputs_2 = inputs[1].to(device)
            optimizer.zero_grad()
            features_1 = net(inputs_1)
            features_2 = net(inputs_1)
            logits, labels = info_nce_loss([features_1, features_2])
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

            pbar.set_description(
                f"epoch {epoch_idx + 1} iter {batch_idx}: train loss {loss.item():.5f}, lr:{get_lr(optimizer)}")
        return train_loss / (batch_idx + 1)

# Fit Model

In [15]:
# Model
net = SimCLR(64)

# Loss & Optimizer & Config
net._config['max_epoch'] = 50

# net._optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
net._optimizer = torch.optim.Adam(net.parameters(), lr=3e-4, weight_decay=0)
net._lossfn = nn.CrossEntropyLoss()
net._lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(net._optimizer, net._config['max_epoch'])

# Set GPU
device = net.auto_gpu()

auto_gpu: using GPU (Tesla P100-PCIE-16GB)


In [16]:
# Fit
net.train()
net.fit(trainset, batch_size=256, workers=2)

epoch 1 iter 195: train loss 0.00183, lr:0.0003: 100%|██████████| 196/196 [01:59<00:00,  1.64it/s]

Epoch 0: Training loss: 0.9489260352684699. Testing loss: Not Provided



epoch 2 iter 195: train loss 0.00084, lr:0.0002997040092642407: 100%|██████████| 196/196 [02:00<00:00,  1.62it/s]

Epoch 1: Training loss: 0.004472448309700537. Testing loss: Not Provided



epoch 3 iter 195: train loss 0.00067, lr:0.0002988172051971717: 100%|██████████| 196/196 [02:01<00:00,  1.61it/s]


...

Epoch 47: Training loss: 0.005229836742617. Testing loss: Not Provided



epoch 49 iter 195: train loss 0.00179, lr:1.1827948028283352e-06: 100%|██████████| 196/196 [02:00<00:00,  1.63it/s]

Epoch 48: Training loss: 0.005206644658607488. Testing loss: Not Provided



epoch 50 iter 195: train loss 0.00115, lr:2.9599073575926616e-07: 100%|██████████| 196/196 [02:00<00:00,  1.62it/s]

Epoch 49: Training loss: 0.00519908163447066. Testing loss: Not Provided


In [17]:
net.backbone.fc[0].in_features

2048

# Use SimCLR to train classifier

Now we have trained the backbone's embedding in constractive learning way.  
We would like to use this pre-trained backbone (exclude fc layers) to train a classifier and do prediction.

In [18]:
class Resnet_constractive(nn.Module):
    def __init__(self, pretrained_model, num_class):
        super(Resnet_constractive, self).__init__()
        fc_in_features = net.backbone.fc[0].in_features
        self.backbone = pretrained_model
        self.backbone.fc = nn.Identity()
        self.fc = nn.Linear(fc_in_features, num_class)
        btorch.utils.trainer.freeze(self.backbone)

    def forward(self, x):
        x = self.backbone(x)
        return self.fc(x)

In [27]:
classifier = Resnet_constractive(net.backbone, 10)
classifier.compile(torch.optim.Adam(classifier.parameters(), lr=3e-4),
                   torch.nn.CrossEntropyLoss())

# Set GPU
device = classifier.auto_gpu()

auto_gpu: using GPU (Tesla P100-PCIE-16GB)


In [37]:
classifier.fit(trainset_clf, validation_data=testset, epochs=20, scoring='accuarcy', batch_size=256)

/opt/conda/lib/python3.7/site-packages/btorch/nn/modules/module.py:237: UserWarning: x might not support <class 'torchvision.datasets.cifar.CIFAR10'>. It will treat x as ``Dataset``.
  f"x might not support {type(x)}. It will treat x as ``Dataset``.")
epoch 1 iter 195: train loss 2.18137, lr:0.0003: 100%|██████████| 196/196 [00:27<00:00,  7.24it/s]


Epoch 0: Training loss: 2.2176359685099856. Testing loss: {'loss': 2.143305603504181, 'score': 0.2169}


epoch 2 iter 195: train loss 2.10966, lr:0.0003: 100%|██████████| 196/196 [00:27<00:00,  7.17it/s]


Epoch 1: Training loss: 2.114233581387267. Testing loss: {'loss': 2.0952429695129395, 'score': 0.2369}


epoch 3 iter 195: train loss 2.01267, lr:0.0003: 100%|██████████| 196/196 [00:27<00:00,  7.24it/s]


...

Epoch 17: Training loss: 1.9359692900764698. Testing loss: {'loss': 1.9559478386878968, 'score': 0.2902}


epoch 19 iter 195: train loss 1.85459, lr:0.0003: 100%|██████████| 196/196 [00:26<00:00,  7.33it/s]


Epoch 18: Training loss: 1.932980623172254. Testing loss: {'loss': 1.9756568343162537, 'score': 0.2856}


epoch 20 iter 195: train loss 1.92412, lr:0.0003: 100%|██████████| 196/196 [00:26<00:00,  7.31it/s]


Epoch 19: Training loss: 1.929997804821754. Testing loss: {'loss': 1.94420458650589, 'score': 0.2963}
